# Notebook pour créer des courbes de lumière
En utilisant les fonctions de calibration et de photmétrie disponibles, on va créer simplement une courbe de lumière à partir des images d'un dossier (correspondant à une cible).

In [ ]:
import os, time
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import TimeSeries
from scipy.ndimage import median_filter

In [ ]:
aster_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
os.listdir(aster_dir)

In [ ]:
for adir in os.listdir(aster_dir):
    asteroids_dir = os.path.join(aster_dir, adir)
    if os.path.isdir(asteroids_dir):
        list_fits = [im for im in sorted(os.listdir(asteroids_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(asteroids_dir, len(list_fits))

In [ ]:
achoice = os.listdir(aster_dir)[0]
aligned = False
asteroids_dir = os.path.join(aster_dir, achoice)
if aligned:
    asteroids_dir = os.path.join(asteroids_dir, "aligned")
list_fits = [im for im in sorted(os.listdir(asteroids_dir)) if ".fits" in im]
list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
rerun = True
use_calib = True
use_sextractor = True

if rerun:
    from tqdm import tqdm
    from astropy.timeseries import TimeSeries
    from astropy.table import vstack

    import warnings
    from astropy.utils.exceptions import AstropyWarning

    warnings.simplefilter("ignore", category=AstropyWarning)

    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(asteroids_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            if use_calib:
                bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

                # Get information from FITS header
                sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)
                # print(sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y)

                # Run calibration
                dico_calib = calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, path_to_bias_dir="", use_bias=False, override_date_check=True, max_days=7, speedup=True, verbose=False, write_tmp=True)
                red_sci_image = dico_calib["path"]
            else:
                red_sci_image = fits_sci_image

            # Analysis
            # hdu = fits.open(red_sci_image)[0]
            im_dir = os.path.abspath(os.path.dirname(red_sci_image))
            im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
            if loc == 0:
                if use_sextractor:
                    sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME tmp_sources.cat -CATALOG_TYPE FITS_1.0 -GAIN 0.932 -VERBOSE_TYPE QUIET"
                    os.system(sex_cmd)
                    cat_tab = Table.read("tmp_sources.cat")
                    cat_tab.rename_column("X_IMAGE", "xcentroid")
                    cat_tab.rename_column("Y_IMAGE", "ycentroid")
                    fwhm, _, _ = sigma_clipped_stats(cat_tab["FWHM_IMAGE"])
                else:
                    try:
                        sources = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
                        fwhm = photometry.get_fwhm(red_sci_image, sources)
                    except RuntimeError:
                        fwhm = 10
            phot_target_table = photometry.query_named_sso_photometry(red_sci_image, fwhm, verbose=False)
            # phot_target_table.pprint()

            if loc == 0:
                concat_ts = phot_target_table
            else:
                try:
                    concat_ts = vstack([concat_ts, phot_target_table])
                except ValueError:
                    pass
    try:
        if use_calib:
            if aligned:
                concat_ts.write(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
            else:
                concat_ts.write(f"{asteroids_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
        else:
            if aligned:
                concat_ts.write(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
            else:
                concat_ts.write(f"{asteroids_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
    except OSError:
        time.sleep(5)
        if use_calib:
            if aligned:
                concat_ts.write(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
            else:
                concat_ts.write(f"{asteroids_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
        else:
            if aligned:
                concat_ts.write(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
            else:
                concat_ts.write(f"{asteroids_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
    # concat_ts.pprint()
    # plt.errorbar(phot_target_table["Epoch"], phot_target_table["aper_sum_bkgsub"], yerr=phot_target_table["noise"])
else:
    if use_calib:
        if aligned:
            concat_ts = Table.read(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits")
        else:
            concat_ts = Table.read(f"{asteroids_dir.split('-')[-1]}_CAL.fits", format="fits")
    else:
        if aligned:
            concat_ts = Table.read(f"{asteroids_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits")
        else:
            concat_ts = Table.read(f"{asteroids_dir.split('-')[-1]}.fits", format="fits")

In [ ]:
concat_ts

In [ ]:
concat_ts["time"].shape

In [ ]:
#!pip install ipympl
# %matplotlib widget
sel = concat_ts["aper_sum_bkgsub"] > 0
concat_ts = concat_ts[sel]
plt.scatter(concat_ts.time.datetime64, concat_ts["aper_sum_bkgsub"])
plt.tick_params(axis="x", labelrotation=90)
plt.ylim(-100, 6000)

In [ ]:
from astropy.stats import sigma_clipped_stats

mean, med, sig = sigma_clipped_stats(concat_ts["aper_sum_bkgsub"])
# %matplotlib inline

_sel = concat_ts["SNR"] > 2
plt.plot(concat_ts[_sel].time.datetime64, concat_ts[_sel]["aper_sum_bkgsub"])
plt.tick_params(axis="x", labelrotation=90)
plt.ylim(-100, 6000)